<a href="https://colab.research.google.com/github/b08202011/2023-ml-final/blob/main/ml_final_project_L1losstest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
!gdown 15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
!gdown 187gijxd4T5yuZe_g2K9UNepx7MT1CnAu

Downloading...
From: https://drive.google.com/uc?id=1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
To: /content/train.csv
100% 23.3M/23.3M [00:00<00:00, 53.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
To: /content/test.csv
100% 8.88M/8.88M [00:00<00:00, 50.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=187gijxd4T5yuZe_g2K9UNepx7MT1CnAu
To: /content/sample_submission.csv
100% 50.5k/50.5k [00:00<00:00, 102MB/s]


In [2]:
import csv
import numpy as np
with open('train.csv',newline='') as csvfile:
  train = csv.reader(csvfile)
  train = list(train)

In [3]:
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,23]
#可以選要用那些特徵，0是我們要predict的
#只選數值或true fale 但且沒有選id
feature = []
for i in range(1,len(train)):
  feature.append([])
  for j in pick:
    feature[i-1].append(train[i][j])

for i in range(len(feature)):
  for j in range(len(feature[i])):
    if(feature[i][j]!='' and feature[i][j]!='False' and feature[i][j]!= 'True'):
      feature[i][j] = float(feature[i][j])
    elif(feature[i][j] == 'False'):
      feature[i][j] = 0
    elif(feature[i][j] == 'True'):
      feature[i][j] = 1
    else:
      feature[i][j] = np.nan
print(len(feature))

17170


In [4]:
!pip install ycimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
feature = np.array(feature)
from ycimpute.imputer import knnimput
feature = knnimput.KNN(k=4).complete(feature) 
##可以修改這裡 使用不同的data填補方式

Imputing row 1/17170 with 1 missing, elapsed time: 82.766
Imputing row 101/17170 with 1 missing, elapsed time: 82.787
Imputing row 201/17170 with 0 missing, elapsed time: 82.815
Imputing row 301/17170 with 2 missing, elapsed time: 82.840
Imputing row 401/17170 with 1 missing, elapsed time: 82.862
Imputing row 501/17170 with 1 missing, elapsed time: 82.883
Imputing row 601/17170 with 4 missing, elapsed time: 82.905
Imputing row 701/17170 with 2 missing, elapsed time: 82.927
Imputing row 801/17170 with 2 missing, elapsed time: 82.951
Imputing row 901/17170 with 3 missing, elapsed time: 82.977
Imputing row 1001/17170 with 0 missing, elapsed time: 83.001
Imputing row 1101/17170 with 0 missing, elapsed time: 83.020
Imputing row 1201/17170 with 2 missing, elapsed time: 83.042
Imputing row 1301/17170 with 3 missing, elapsed time: 83.064
Imputing row 1401/17170 with 3 missing, elapsed time: 83.084
Imputing row 1501/17170 with 3 missing, elapsed time: 83.107
Imputing row 1601/17170 with 3 missi

In [6]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv
from sklearn.feature_selection import SelectKBest
# For Progress Bar
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_regression
# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

In [7]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)
                                 
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

##Dataloader

In [8]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Model

In [9]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(16, 4),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(4, 1),           
        )

    def forward(self, x):
        #x = x.view(-1, 16)
        x = self.layers(x)
        x = x.squeeze(-1) # (B, 1) -> (B)
        return x

## Training

In [15]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.L1Loss() # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), lr= config['learning_rate'] , weight_decay=0.01, amsgrad=False) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []


        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            #print(pred.tolist())       
            #print(y.tolist())
                 
            loss = criterion(pred, y)
            #print(loss)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            
            loss_record.append(loss.detach().item())
            
            

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## config

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.25,   # validation_size = train_size * valid_ratio
    'n_epochs': 30000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-3,              
    'early_stop': 5000,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


In [17]:
same_seed(config['seed'])
with open('test.csv',newline='') as csvfile:
  testcsv = csv.reader(csvfile)
  testcsv = list(testcsv)
test = []
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,22]

for i in range(1,len(testcsv)):
  test.append([])
  for j in pick:
    test[i-1].append(testcsv[i][j])
for i in range(len(test)):
  for j in range(len(test[i])):
    if(test[i][j]!='' and test[i][j]!='False' and test[i][j]!= 'True'):
      test[i][j] = float(test[i][j])
    elif(test[i][j] == 'False'):
      test[i][j] = 0
    elif(test[i][j] == 'True'):
      test[i][j] = 1
    else:
      test[i][j] = np.nan
test_data = knnimput.KNN(k=4).complete(np.array(test))
train_data= feature
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

x_train, x_valid, x_test, y_train, y_valid = train_data[...,1:], valid_data[...,1:], test_data, train_data[...,0], valid_data[...,0]
# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")


# Select features



# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset  = COVID19Dataset(x_train, y_train)

valid_dataset = COVID19Dataset(x_valid, y_valid)
print(train_dataset.__getitem__(0))
test_dataset = COVID19Dataset(x_test)
# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

Imputing row 1/6315 with 1 missing, elapsed time: 11.382
Imputing row 101/6315 with 2 missing, elapsed time: 11.392
Imputing row 201/6315 with 3 missing, elapsed time: 11.402
Imputing row 301/6315 with 2 missing, elapsed time: 11.413
Imputing row 401/6315 with 4 missing, elapsed time: 11.422
Imputing row 501/6315 with 1 missing, elapsed time: 11.432
Imputing row 601/6315 with 1 missing, elapsed time: 11.442
Imputing row 701/6315 with 2 missing, elapsed time: 11.452
Imputing row 801/6315 with 3 missing, elapsed time: 11.462
Imputing row 901/6315 with 0 missing, elapsed time: 11.473
Imputing row 1001/6315 with 3 missing, elapsed time: 11.482
Imputing row 1101/6315 with 1 missing, elapsed time: 11.493
Imputing row 1201/6315 with 5 missing, elapsed time: 11.503
Imputing row 1301/6315 with 3 missing, elapsed time: 11.513
Imputing row 1401/6315 with 2 missing, elapsed time: 11.522
Imputing row 1501/6315 with 1 missing, elapsed time: 11.532
Imputing row 1601/6315 with 3 missing, elapsed time:

## Training and testing

In [18]:


print(x_train.shape[1])
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)





16
Epoch [1/30000]: Train loss: 4.6153, Valid loss: 4.5819
Saving model with loss 4.582...
Epoch [2/30000]: Train loss: 4.4284, Valid loss: 4.3537
Saving model with loss 4.354...
Epoch [3/30000]: Train loss: 4.2935, Valid loss: 4.2123
Saving model with loss 4.212...
Epoch [4/30000]: Train loss: 4.1757, Valid loss: 4.0928
Saving model with loss 4.093...
Epoch [5/30000]: Train loss: 4.0583, Valid loss: 4.0102
Saving model with loss 4.010...
Epoch [6/30000]: Train loss: 3.9263, Valid loss: 3.8543
Saving model with loss 3.854...
Epoch [7/30000]: Train loss: 3.7776, Valid loss: 3.6911
Saving model with loss 3.691...
Epoch [8/30000]: Train loss: 3.6254, Valid loss: 3.5048
Saving model with loss 3.505...
Epoch [9/30000]: Train loss: 3.4940, Valid loss: 3.4107
Saving model with loss 3.411...
Epoch [10/30000]: Train loss: 3.3587, Valid loss: 3.2347
Saving model with loss 3.235...
Epoch [11/30000]: Train loss: 3.2492, Valid loss: 3.0864
Saving model with loss 3.086...
Epoch [12/30000]: Train los

KeyboardInterrupt: ignored

In [20]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'Danceability'])
        for i, p in enumerate(preds):
            writer.writerow([i+17170, p])
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
print(preds)
save_pred(preds, 'sample_submission.csv')
from google.colab import files
files.download('sample_submission.csv')

100%|██████████| 25/25 [00:00<00:00, 727.27it/s]

[5.980056  4.6319475 4.0384264 ... 4.257544  5.5722847 5.7910714]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>